In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from gql import Client, gql
from gql.transport.httpx import HTTPXTransport
import os
from tqdm.auto import tqdm
import pandas as pd

In [3]:
GQL_CLIENT_TIMEOUT = 60
RAW_DATA_PATH = '../data/raw/PL-22-23'


In [4]:
from gandula.providers.pff.api.api_client import execute_query, get_client

api_url = os.getenv('api_url')
api_key = os.getenv('api_key')
client = get_client(api_url, api_key)

# Queries

In [13]:
get_players = """
query rosters($gameId: ID!)  {
  rosters(gameId: $gameId) {
    positionGroupType
    player {
      id
      firstName
      lastName
      nickname
    }
    shirtNumber
    team {
      id
    }
  }
}
"""

In [14]:
get_game = """
query game($id: ID!) {
  game(id: $id) {
     homeTeam {
      id
      name
    }
    awayTeam {
      id
      name
    }
    competition {
      id
      name
    }
    date
    homeTeamStartLeft
    id
    season
    week
  }
}
"""

In [15]:
get_teams = """
query game($id: ID!) {
  game(id: $id) {
    id
    homeTeam  {
      id
      name
      shortName
      country
      kits {
        name 
        primaryColor
        primaryTextColor
        secondaryColor
        secondaryTextColor
        updatedAt 
      }
    }
  }
}

"""

In [16]:
games = os.listdir(RAW_DATA_PATH)
game_ids = [game.split('.')[0] for game in games]
print(f"Games to process: {len(game_ids)}")

Games to process: 190


In [19]:
games = []
teams = []
team_ids = []
players = []
player_ids = []

for game_id in tqdm(game_ids, desc='Processing games', total=len(game_ids)):
    variables = {'id': int(game_id)}
    player_variables = {'gameId': int(game_id)}

    game_result = client.execute(
            gql(get_game),
            variable_values=variables,
            serialize_variables=True,
            parse_result=True,
        )
    
    games.append(game_result['game'])

    teams_result = client.execute(
            gql(get_teams),
            variable_values=variables,
            serialize_variables=True,
            parse_result=True,
        )
    
    if teams_result['game']['homeTeam']['id'] not in team_ids:
        teams.append(teams_result['game'])
        team_ids.append(teams_result['game']['homeTeam']['id'])

    players_result = client.execute(
            gql(get_players),
            variable_values=player_variables,
            serialize_variables=True,
            parse_result=True,
        )
    
    players.append(players_result['rosters'])

Processing games:   0%|          | 0/190 [00:00<?, ?it/s]

In [39]:
def games_to_df(games):
    
    games_list = []

    for game in games:

        games_list.append({
            'match_id': int(game['id']),
            'season': game['season'],
            'week': int(game['week']),
            'date': pd.to_datetime(game['date']),
            'home_team_id': int(game['homeTeam']['id']),
            'home_team_name': game['homeTeam']['name'],
            'away_team_id': int(game['awayTeam']['id']),
            'away_team_name': game['awayTeam']['name'],
            'competition_id': int(game['competition']['id']),
            'competition_name': game['competition']['name'],
            'home_team_start_left': game['homeTeamStartLeft']

        })

    return pd.DataFrame(games_list).sort_values(by='match_id').reset_index(drop=True)

In [40]:
def teams_to_df(teams):
    
    teams_list = []

    for team in teams:

        kits = team['homeTeam']['kits']  # lista de dicionários

        # 1) Filtrar apenas kits com name=='home'
        home_kits = [k for k in kits if k['name'].lower() == 'home']
        
        if home_kits:
            # Ordenar pelos updatedAt (ISO 8601) em ordem decrescente
            home_kits.sort(key=lambda x: x['updatedAt'], reverse=True)
            chosen_kit = home_kits[0]  # mais recente
        else:
            # 2) Se não existir kit 'home', usar o kit mais recente entre todos
            kits.sort(key=lambda x: x['updatedAt'], reverse=True)
            chosen_kit = kits[0]  # mais recente no geral

        primary_color = chosen_kit['primaryColor']
        secondary_color = chosen_kit['secondaryColor']

        teams_list.append({
            'team_id': int(team['homeTeam']['id']),
            'team_name': team['homeTeam']['name'],
            'team_short_name': team['homeTeam']['shortName'],
            'team_country': team['homeTeam']['country'],
            'primary_color': primary_color,
            'secondary_color': secondary_color
        })

    return pd.DataFrame(teams_list).drop_duplicates(subset=['team_id']).sort_values('team_id').reset_index(drop=True)

In [51]:
def players_to_df(players_rosters):
    
    players_list = []

    for roster in players_rosters:

        for player in roster:

            players_list.append({
                'player_id': int(player['player']['id']),
                'first_name': player['player']['firstName'],
                'last_name': player['player']['lastName'],
                'nickname': player['player']['nickname'],
                'team_id': int(player['team']['id']),
                'position': player['positionGroupType'],
                'shirt_number': player['shirtNumber']
            })

    return pd.DataFrame(players_list).drop_duplicates(subset=['player_id','team_id']).sort_values('player_id').reset_index(drop=True)

In [42]:
games_df = games_to_df(games)
games_df 

,match_id,season,week,date,home_team_id,home_team_name,away_team_id,away_team_name,competition_id,competition_name,home_team_start_left
0,4436,2022-2023,1,2022-08-05 19:00:00,7,Crystal Palace,2,Arsenal,1,Premier League,True
1,4437,2022-2023,1,2022-08-06 11:30:00,54,Fulham,10,Liverpool,1,Premier League,True
2,4438,2022-2023,1,2022-08-06 14:00:00,1,AFC Bournemouth,3,Aston Villa,1,Premier League,False
3,4439,2022-2023,1,2022-08-06 14:00:00,55,Leeds United,20,Wolverhampton Wanderers,1,Premier League,True
4,4440,2022-2023,1,2022-08-07 13:00:00,9,Leicester City,119,Brentford,1,Premier League,True
...,...,...,...,...,...,...,...,...,...,...,...
185,4621,2022-2023,19,2023-01-03 19:45:00,8,Everton,4,Brighton & Hove Albion,1,Premier League,True
186,4622,2022-2023,19,2023-01-04 19:45:00,55,Leeds United,19,West Ham,1,Premier League,True
187,4623,2022-2023,19,2023-01-03 19:45:00,9,Leicester City,54,Fulham,1,Premier League,True
188,4624,2022-2023,19,2023-01-03 20:00:00,12,Manchester United,1,AFC Bournemouth,1,Premier League,True


In [43]:
teams_df = teams_to_df(teams)
teams_df 

,team_id,team_name,team_short_name,team_country,primary_color,secondary_color
0,1,AFC Bournemouth,BOU,England,#E20613,#000000
1,2,Arsenal,ARS,England,#E10B17,#FFFFFF
2,3,Aston Villa,AVL,England,#660E36,#93BDE4
3,4,Brighton & Hove Albion,BHA,England,#0A55A2,#FFFFFF
4,6,Chelsea,CHE,England,#184890,#1D1D1D
5,7,Crystal Palace,CRY,England,#DF2443,#3258A3
6,8,Everton,EVE,England,#0B478F,#FFFFFF
7,9,Leicester City,LCI,England,#233A80,#FFFFFF
8,10,Liverpool,LIV,England,#D01317,#FFFFFF
9,11,Manchester City,MCI,England,#6CABDD,#FFFFFF


In [52]:
players_df = players_to_df(players)
players_df 

,player_id,first_name,last_name,nickname,team_id,position,shirt_number
0,1,Harry,Kane,Harry Kane,17,CF,10
1,6,Ederson,Santana de Moraes,Ederson,11,GK,31
2,9,Aymeric,Laporte,Aymeric Laporte,11,LCB,14
3,10,John,Stones,John Stones,11,RCB,5
4,13,Tosin,Adarabioyo,Tosin Adarabioyo,54,RCB,4
...,...,...,...,...,...,...,...
439,13447,Juan,Larios,Juan Larios,16,LB,28
440,13638,Wilfried,Gnonto,Wilfried Gnonto,55,LW,29
441,14238,Carlos,Alcaraz,Carlos Alcaraz,16,AM,26
442,14289,Danilo,dos Santos de Oliveira,Danilo,221,CM,28


# Export CSV

In [53]:
games_df.to_csv('../data/csv/games.csv', index=False)
teams_df.to_csv('../data/csv/teams.csv', index=False)
players_df.to_csv('../data/csv/players.csv', index=False)